In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_dataput_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

BATCH_SIZE = 100
INPUT_MODE = 784
OUTPUT_MODE = 10
LAYER1_MDOE = 500

LEARNING_RATE_BASE = .8
LEARNING_RATE_DECAY = .99

REGULARIZATION_RATE = 0.0001

TRAINING_STEPS = 5000

MOVING_AVERAGE_DECAY = .9999

def inference(input_tensor, avg_class, weights1,biases1, weights2, biases2):
    
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        return tf.matmul(layer1, weights2) + biases2
    
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
def train(mnist):
    x = tf.placeholder(tf.float32, shape=[None, INPUT_MODE],name="input_x")
    y_ = tf.placeholder(tf.float32, shape=[None, OUTPUT_MODE],name="input_y")
    
    weights1 = tf.Variable(tf.truncated_normal(shape=[INPUT_MODE, LAYER1_MDOE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_MDOE]))
    
    weights2 = tf.Variable(tf.truncated_normal(shape=[LAYER1_MDOE, OUTPUT_MODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_MODE]))
    
    y = inference(x, None, weights1=weights1, biases1=biases1, weights2=weights2, biases2=biases2)
    
    global_step = tf.Variable(0, trainable=False)
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    
    average_y = inference(x, variable_averages, weights1=weights1, biases1=biases1, weights2=weights2,biases2=biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.arg_max(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weights1) + regularizer(weights2)
    
    loss = cross_entropy_mean + regularization
    
    learning_rate = tf.train.exponential_decay(
                    LEARNING_RATE_BASE,
                    global_step,
                    mnist.train.num_examples / BATCH_SIZE,
                    LEARNING_RATE_DECAY,
                    staircase=True
                    )
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name="train")
    
    corrent_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(corrent_prediction, tf.float32))
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        
        validate_feed = {
            x:mnist.validation.images,
            y_:mnist.validation.labels
        }
        
        test_feed = {
            x: mnist.test.images,
            y_: mnist.test.labels
        }
        
        
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print(f'After {i} training steps, validation accuracy is {validate_acc}')
            
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,  feed_dict={x:xs, y_:ys})
            
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print(f'after {TRAINING_STEPS} steps, test accuracy is {test_acc}')


def main(avg=None):
    train(mnist)
    
if __name__ == "__main__":
    main()

After 0 training steps, validation accuracy is 0.09640000015497208
After 1000 training steps, validation accuracy is 0.9728000164031982
After 2000 training steps, validation accuracy is 0.9782000184059143
After 3000 training steps, validation accuracy is 0.9801999926567078
After 4000 training steps, validation accuracy is 0.9764000177383423
after 5000 steps, test accuracy is 0.9825999736785889
